<a href="https://colab.research.google.com/github/PranavSuresh525/AI-ML-Projects/blob/main/AI_Integration_in_Finance/Finance_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
!pip install -q -U --no-warn-conflicts \
    langchain-huggingface \
    langchain-google-genai \
    langgraph \
    yfinance \
    transformers \
    accelerate \
    duckduckgo-search \
    langchain-text-splitters \
    langchain-chroma \
    langchain-community \
    ddgs\
    newsapi-python
import numpy as np
import requests
import os
import re
import json
import time
from newsapi import NewsApiClient
import operator
from datetime import datetime, timedelta
from typing import TypedDict, List, Annotated, Dict, Any, Optional
import yfinance as yf
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END, START
from IPython.display import HTML, Markdown
from google.colab import userdata

In [110]:
## the local llm whixh avoids unecessary calls from gemini as there is a API limit
local_pipe = pipeline("text2text-generation", model="google/flan-t5-large", max_new_tokens=256, device_map="auto")
local_llm = HuggingFacePipeline(pipeline=local_pipe)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_TOKEN')
llm = ChatGoogleGenerativeAI(
    model="gemma-3-12b-it",
    temperature=0.2,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)
# WEB SEARCH: Used by the local node to find tickers
search_tool = DuckDuckGoSearchRun()
_ticker_cache={}

Device set to use cpu


In [111]:
def smart_llm_invoke(prompt):
  #"""Try Gemini first, fallback to local LLM if it fails"""
  try:
      response = llm.invoke([HumanMessage(content=prompt)])
      return response.content.strip()
  except Exception as e:
      print(f"Error from Gemini: {e}")
      response = local_llm.invoke(prompt)  # Remove the [HumanMessage(...)] wrapper
      return response.strip()

In [112]:
# a basic function to get all the info related to a stock
def get_stock_price(ticker: str):
  try:
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period='1y')

    if not info or not info.get('symbol') or history.empty:
      return {"error": f"No data found or invalid ticker for {ticker}"}

    return{
            'ticker': ticker,
            'current_price': info.get('currentPrice', 0),
            'previous_close': info.get('previousClose', 0),
            'day_high': info.get('dayHigh', 0),
            'day_low': info.get('dayLow', 0),
            'volume': info.get('volume', 0),
            'market_cap': info.get('marketCap', 0),
            'company_name': info.get('longName', ticker),
            'pe_ratio': info.get('trailingPE', 0),
            'dividend_yield': info.get('dividendYield', 0),
            'target_mean_price': info.get('targetMeanPrice', 0),
            'recommendation_key': info.get('recommendationKey', 'N/A'),
            '50_day_average': history['Close'].rolling(window=50).mean().iloc[-1] if len(history) >= 50 else 0,
            '200_day_average': history['Close'].rolling(window=200).mean().iloc[-1] if len(history) >= 200 else 0,
            'price_history': history['Close'].tail(30).to_list()
        }
  except Exception as e:
        return {"error": f"Failed to fetch data for {ticker}: {str(e)}"}

In [113]:
# a function which accesses a pandas data frame and gets the latest stock price of the company-'item' here
def get_recent_data(df, items):
  if df.empty:
    return {}
  recent_data={}
  for item in items:
    try:
      if item in df.index:
          value = df.loc[item].iloc[0]
          recent_data[item] = float(value) if value is not None else 0
      else:
          recent_data[item] = 0
    except:
      recent_data[item] = 0


In [114]:
# a function which basically gets the latest info (listed below) from yahoo finance, uses the period here to know how long to look for
def fetch_financial_statements(ticker: str, period: str)->dict:
  try:
    stock=yf.Ticker(ticker)
    if not stock.info or not stock.info.get('symbol'):
      return {'error': f'Invalid or no data for ticker {ticker}'}

    if period=='quaterly':
      balance_sheet=stock.quarterly_balance_sheet
      income_statement=stock.quarterly_income_stmt
      cash_flow=stock.quarterly_cashflow
    else:
      balance_sheet=stock.balance_sheet
      income_statement=stock.income_stmt
      cash_flow=stock.cashflow
    return{
        'balance_sheet': get_recent_data(balance_sheet, ['Total Cash', 'Total Debt']),
        'income_statement': get_recent_data(income_statement, ['Total Revenue', 'Gross Profit']),
        'cash_flow': get_recent_data(cash_flow, ['Net Cash Flow']),
        'period': period
    }
  except Exception as e:
    return {"error": f"Failed to fetch data for {ticker}: {str(e)}"}

In [115]:
# a simple validate function that cross checks with yf to see of the ticker exist
def validate_ticker(potential_ticker: str):
  try:
    stock = yf.Ticker(potential_ticker)
    info = stock.info
    if info and 'symbol' in info and info.get('symbol'):
      return info['symbol']
    return None
  except Exception as e:
    return None

In [116]:
def analyze_sentiment(text: str):
    if not text:
        return 0.0

    prompt = f"""Analyze the sentiment of this financial news text.
    Return ONLY a number between -1 (very negative) and 1 (very positive).

    Text: {text}
    sentiment:"""

    try:
        response = smart_llm_invoke(prompt)
        sentiment_score = float(response.strip())
        return max(-1, min(1, sentiment_score))
    except:
        pass

    # Fallback to keyword-based analysis
    positive_words = [
        'gain', 'gains', 'up', 'rise', 'surge', 'rally', 'jump', 'soar',
        'climb', 'boost', 'profit', 'growth', 'outperform', 'beat', 'strong',
        'bullish', 'optimistic', 'confidence', 'upgrade', 'buy', 'positive'
    ]

    negative_words = [
        'loss', 'down', 'drop', 'fall', 'decline', 'plunge', 'crash',
        'tumble', 'slump', 'miss', 'disappoint', 'weak', 'bearish',
        'pessimistic', 'concern', 'worry', 'fear', 'downgrade', 'sell'
    ]

    intensifiers = ['very', 'extremely', 'highly', 'significantly']
    negations = ['not', 'no', 'never', "don't", "doesn't", "won't"]

    words = text.lower().split()
    positive_count = 0
    negative_count = 0

    for i, word in enumerate(words):
        # Check for intensifiers
        multiplier = 1.5 if i > 0 and words[i-1] in intensifiers else 1.0

        # Check for negations (flip sentiment)
        is_negated = i > 0 and words[i-1] in negations

        if word in positive_words:
            if is_negated:
                negative_count += multiplier
            else:
                positive_count += multiplier
        elif word in negative_words:
            if is_negated:
                positive_count += multiplier
            else:
                negative_count += multiplier

    if positive_count + negative_count == 0:
        return 0.0

    return (positive_count - negative_count) / (positive_count + negative_count)

In [117]:
# the heart of the model that connects the model through a RAG pipeline
class NewsRAG:
  def __init__(self, embedding_model="sentence-transformers/all-MiniLM-L6-v2"):
    self.embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model
    )
    self.text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    self.vectorstore = None
  def index_news(self, news_articles: List[Dict]):
    """
    news_articles example:
    {
        "title": "...",
        "content": "...",
        "link": "...",
        "date": "...",
        "source": "..."
    }
    """
    documents = []
    for article in news_articles:
        content = article.get("content", "") or article.get("title", "")
        header=f"Source: {article.get('source')}| Date: {article.get('date')}"
        metadata = {
            "title": article.get("title", ""),
            "link": article.get("link", ""),
            "date": article.get("date", ""),
            "source": article.get("source", "")
        }

        documents.append(
            Document(
                page_content=content,
                metadata=metadata
            )
        )
    if documents:
      splits = self.text_splitter.split_documents(documents)
      self.vectorstore = Chroma.from_documents(
          documents=splits,
          embedding=self.embeddings
      )

  def retrieve_context(self, query: str, k: int = 5) -> List[Dict]:
    if self.vectorstore is None:
      return []
    docs = self.vectorstore.similarity_search(query, k=k)
    return self.__distill_context(query, docs)

  def __distill_context(self, query: str, docs: List[Document]) -> List[Dict]:
    raw_text="\n--\n".join([d.page_content for d in docs])
    distil_prompts=f"""
        Extract ONLY the facts from the news snippets below that directly answer the query: "{query}"
        If the snippets are irrelevant, return "No relevant news found."

        Snippets:
        {raw_text}

        Key Facts:
        """
    response = smart_llm_invoke(distil_prompts)
    return response.strip().split("\n--\n")

In [118]:
class AgentState(TypedDict):
    query: str
    ticker: str
    intent: str
    price_data: dict
    financial_data: dict
    news_articles: Annotated[list, operator.add]
    news_context: Annotated[list, operator.add]
    sentiment_score: float
    analysis: str
    recommendation: str
    messages: list       # Remove Annotated

In [119]:
def intent_classifier(state: AgentState):
    query = state['query']
    query_lower = query.lower()

    # Check for specific milestone questions (like "when did X hit Y")
    if any(word in query_lower for word in ['when did', 'when was']) and any(word in query_lower for word in ['hit', 'reach', 'achieve', 'cross']):
        intent = 'milestone_query'
    elif any(word in query_lower for word in ['why', 'reason', 'cause']):
        intent = 'reason_query'
    elif any(word in query_lower for word in ['when', 'trend', 'history']):
        intent = 'trend_analysis'
    elif any(word in query_lower for word in ['compare', 'vs', 'versus']):
        intent = 'comparison'
    elif any(word in query_lower for word in ['price', 'cost', 'trading at']):
        intent = 'price_query'
    else:
        intent = 'general'

    state['intent'] = intent
    state['messages'].append(f"[Intent Classifier] Intent: {intent}")
    return state

In [120]:
def ticker_extractor(state):
    query = state["query"].strip()
    q_upper = query.upper()

    # 1️⃣ Explicit ticker check
    explicit_ticker_pattern = r'\b[A-Z&]{2,10}\.[A-Z]{1,4}\b|\$[A-Z]{2,5}\b'
    explicit_matches = re.findall(explicit_ticker_pattern, q_upper)

    for raw in explicit_matches:
        ticker = raw.replace("$", "").strip()
        if validate_ticker(ticker):
            state["ticker"] = ticker
            state["messages"].append(f"[Ticker Extractor] Explicit ticker: {ticker}")
            return state

    # 2️⃣ Extract company name
    extract_prompt = f"""Extract ONLY the company name from this query. Return just the company name, nothing else.

Query: {query}

Company name:"""

    company_name = smart_llm_invoke(extract_prompt).strip()

    # 3️⃣ Yahoo Finance Lookup with ranking
    def yahoo_lookup_best_match(search_term):
        """Use Yahoo Finance search and return the most relevant result"""
        try:
            url = "https://query2.finance.yahoo.com/v1/finance/search"
            params = {
                'q': search_term,
                'quotesCount': 10,  # Get more results
                'newsCount': 0,
                'enableFuzzyQuery': False,
                'quotesQueryId': 'tss_match_phrase_query'
            }
            headers = {'User-Agent': 'Mozilla/5.0'}

            response = requests.get(url, params=params, headers=headers, timeout=5)
            data = response.json()

            quotes = data.get('quotes', [])
            if not quotes:
                return None

            # Score each result for relevance
            def score_result(quote):
                score = 0
                symbol = quote.get('symbol', '')
                name = quote.get('longname', '') or quote.get('shortname', '')
                exchange = quote.get('exchange', '')
                quote_type = quote.get('quoteType', '')

                # Prefer exact company name matches
                if search_term.lower() in name.lower():
                    score += 100

                # Prefer EQUITY over other types
                if quote_type == 'EQUITY':
                    score += 50

                # Prefer major exchanges
                if exchange in ['NSI', 'BSE', 'NMS', 'NYQ']:
                    score += 30

                # Penalize subsidiaries/specific divisions (usually longer names)
                if len(name) > len(search_term) + 20:
                    score -= 20

                # Prefer shorter, cleaner ticker symbols
                if len(symbol) <= 10:
                    score += 10

                # Prefer symbols without special chars (except .NS, .BO)
                base_symbol = symbol.split('.')[0]
                if base_symbol.isalpha():
                    score += 10

                return score

            # Rank results
            ranked = sorted(quotes, key=score_result, reverse=True)

            # Return top match
            return ranked[0].get('symbol')

        except Exception as e:
            print(f"Yahoo lookup failed: {e}")
            return None

    # Try Yahoo lookup
    ticker = yahoo_lookup_best_match(company_name)

    if ticker and validate_ticker(ticker):
        state["ticker"] = ticker
        state["messages"].append(f"[Ticker Extractor] Found via Yahoo: {ticker}")
        return state

    # 4️⃣ LLM fallback with disambiguation
    llm_prompt = f"""What is the PRIMARY/MAIN stock ticker symbol for "{company_name}"?

IMPORTANT: If there are multiple companies with similar names, return the LARGEST/MOST WELL-KNOWN one.

Examples:
- "Mahindra" → M&M.NS (Mahindra & Mahindra, the main automobile company, NOT subsidiaries)
- "Tata" → TATAMOTORS.NS (the main auto company)
- "Adani" → ADANI.NS (Adani Enterprises, the flagship)
- "Apple" → AAPL (the main company)

For Indian companies: use .NS suffix
For US companies: no suffix

Company: {company_name}

Main ticker symbol:"""

    ticker_response = smart_llm_invoke(llm_prompt).strip().upper()

    # Extract ticker
    match = re.search(r'\b[A-Z&]{1,15}\.(?:NS|BO)\b|\b[A-Z]{1,5}\b', ticker_response)
    ticker = match.group(0) if match else ticker_response

    # Validate
    if ticker and validate_ticker(ticker):
        state["ticker"] = ticker
        state["messages"].append(f"[Ticker Extractor] LLM resolved: {ticker}")
        return state

    # Failed
    state["ticker"] = ""
    state["messages"].append(f"[Ticker Extractor] Could not find ticker for {company_name}")

    return state

In [121]:
def price_fetcher(state: AgentState):
  ticker=state['ticker']
  if ticker:
    price_data=get_stock_price(ticker)
    state['price_data']=price_data
    state['messages'].append(f"[Price Fetcher] Price Data: {price_data}")
  else:
    state['price_data']={}
    state['messages'].append(f"[Price Fetcher] No price data found")
  return state

In [122]:
def financial_fetcher(state: AgentState):
  ticker=state['ticker']
  intent=state['intent']
  if intent in ['general', 'comparison'] and ticker:
    financial_data=fetch_financial_statements(ticker, 'annual')
    state['financial_data']=financial_data
    state['messages'].append(f"[Financial Fetcher] Financial Data: {financial_data}")
  else:
    state['financial_data']={}
    state['messages'].append(f"[Financial Fetcher] No financial data found")
  return state

In [123]:
def news_fetcher(state: AgentState):
    """Fetches news articles using NewsAPI"""
    ticker = state.get('ticker')

    if not ticker:
        state['news_articles'] = []
        state.setdefault('messages', []).append("[News Fetcher] No ticker provided")
        return state

    import traceback
    from datetime import datetime, timedelta

    try:
        # Initialize NewsAPI
        try:
            api_key = userdata.get('NEWSAPI_KEY')
            newsapi = NewsApiClient(api_key=api_key)
        except Exception as e:
            state['news_articles'] = []
            state.setdefault('messages', []).append(f"[News Fetcher] API initialization failed: {e}")
            return state

        # Get company info
        clean_ticker = ticker.split('.')[0]
        price_data = state.get('price_data', {})
        company_name = price_data.get('company_name', clean_ticker)

        # Calculate date range
        intent = state.get('intent', '')
        from_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

        articles = []
        seen_urls = set()

        # Try multiple query strategies
        queries = [
            company_name,
            f"{clean_ticker}",
            f"{company_name} stock",
            f"{company_name} India",
        ]

        for query in queries:
            try:
                # Make API call
                response = newsapi.get_everything(
                    q=query,
                    from_param=from_date,
                    language='en',
                    sort_by='publishedAt',
                    page_size=20
                )

                status = response.get('status')
                news_items = response.get('articles', [])

                if status != 'ok':
                    continue

                # Process articles
                for item in news_items:
                    url = item.get('url', '')

                    if not url or url in seen_urls:
                        continue
                    seen_urls.add(url)

                    # Extract source
                    source_obj = item.get('source', {})
                    source_name = source_obj.get('name', '') if isinstance(source_obj, dict) else str(source_obj)

                    # Content
                    content = item.get('description') or item.get('content') or item.get('title', '')

                    # Date
                    published_date = item.get('publishedAt', '')
                    try:
                        date_obj = datetime.fromisoformat(published_date.replace('Z', '+00:00'))
                        formatted_date = date_obj.strftime('%Y-%m-%d')
                    except:
                        formatted_date = published_date

                    articles.append({
                        'title': item.get('title', ''),
                        'content': content,
                        'link': url,
                        'date': formatted_date,
                        'source': source_name
                    })

                # Stop if we have enough
                if len(articles) >= 20:
                    break

            except Exception as e:
                continue

        # Update state
        state['news_articles'] = articles
        state.setdefault('messages', [])
        state['messages'].append(f"[News Fetcher] Found {len(articles)} articles for {ticker}")

    except Exception as e:
        state['news_articles'] = []
        state.setdefault('messages', []).append(f"[News Fetcher] Error: {str(e)}")

    return state

In [124]:
def news_analyzer(state: AgentState):
  print(f"[NEWS ANALYZER DEBUG] Received {len(state.get('news_articles', []))} articles")
  news_articles = state['news_articles']
  if not news_articles:
    state['news_context']=[]
    state['messages'].append(f"[News Analyzer] No news articles to analyze")
    return state
  rag=NewsRAG()
  rag.index_news(news_articles)

  context=state['news_context']
  if not context:
    context=rag.retrieve_context(state['query'])
  state['news_context']=context

  sentiment=analyze_sentiment("\n--\n".join(context))
  state['sentiment_score']=sentiment

  state['messages'].append(f"[News Analyzer] Sentiment Score: {sentiment}")
  return state

In [125]:
def response_generator(state: AgentState):
    """Generate final response based on all gathered data"""
    query = state['query']
    ticker = state['ticker']
    intent = state['intent']
    price_data = state['price_data']
    news_context = state['news_context']
    news_articles = state.get('news_articles', [])
    sentiment = state['sentiment_score']
    symbol = fetch_currency(ticker)[0]

    # Build context for LLM
    context_parts = []

    if price_data and 'error' not in price_data:
        context_parts.append(f"""
📈 Stock Data for {ticker}

💰 Price & Trading
- Current Price: {symbol}{price_data.get('current_price', 0):.2f}
- Previous Close: {symbol}{price_data.get('previous_close', 0):.2f}
- Open: {symbol}{price_data.get('open', 0):.2f}
- Day High / Low: {symbol}{price_data.get('day_high', 0):.2f} / ${price_data.get('day_low', 0):.2f}
- 52-Week High / Low: {symbol}{price_data.get('52_week_high', 0):.2f} / ${price_data.get('52_week_low', 0):.2f}
- Volume: {price_data.get('volume', 0):,}
- Avg Volume (3M): {price_data.get('avg_volume', 0):,}

📊 Averages & Momentum
- 50-Day Avg: {symbol}{price_data.get('50_day_average', 0):.2f}
- 200-Day Avg: {symbol}{price_data.get('200_day_average', 0):.2f}
- Beta (Volatility): {price_data.get('beta', 'N/A')}

🏦 Valuation
- Market Cap: {symbol}{price_data.get('market_cap', 0):,.0f}
- Enterprise Value: {symbol}{price_data.get('enterprise_value', 0):,.0f}
- P/E Ratio (TTM): {price_data.get('pe_ratio', 'N/A')}
- Forward P/E: {price_data.get('forward_pe', 'N/A')}
- PEG Ratio: {price_data.get('peg_ratio', 'N/A')}
- Price to Book: {price_data.get('price_to_book', 'N/A')}

📉 Financial Health
- Revenue (TTM): {symbol}{price_data.get('revenue', 0):,.0f}
- Revenue Growth (YoY): {price_data.get('revenue_growth', 'N/A')}
- Gross Margin: {price_data.get('gross_margin', 'N/A')}
- Operating Margin: {price_data.get('operating_margin', 'N/A')}
- Net Margin: {price_data.get('profit_margin', 'N/A')}
- Free Cash Flow: {symbol}{price_data.get('free_cashflow', 0):,.0f}
- Debt to Equity: {price_data.get('debt_to_equity', 'N/A')}
- Return on Equity (ROE): {price_data.get('roe', 'N/A')}

💸 Dividends
- Dividend Rate: {symbol}{price_data.get('dividend_rate', 0):.2f}
- Dividend Yield: {price_data.get('dividend_yield', 'N/A')}
- Payout Ratio: {price_data.get('payout_ratio', 'N/A')}

🧠 Analyst View
- Recommendation: {price_data.get('recommendation_key', 'N/A')}
- Target Mean Price: {symbol}{price_data.get('target_mean_price', 0):.2f}
- Target High / Low: {symbol}{price_data.get('target_high_price', 0):.2f} / ${price_data.get('target_low_price', 0):.2f}
- Number of Analysts: {price_data.get('number_of_analyst_opinions', 'N/A')}

🕒 Trading Info
- Currency: {price_data.get('currency', 'USD')}
- Exchange: {price_data.get('exchange', 'N/A')}
- Market State: {price_data.get('market_state', 'N/A')}
""")

    if news_context:
        context_parts.append(f"Recent News Context:\n" + "\n".join(news_context))

    context_parts.append(f"Overall Sentiment Score: {sentiment:.2f} (-1 = very negative, 1 = very positive)")

    full_context = "\n\n".join(context_parts)

    # Create prompt based on intent
    if intent == 'reason_query':
        prompt = f"""You are a financial analyst. Based on the data below, explain WHY {ticker} stock moved recently.

{full_context}

User Question: {query}

Instructions:
- Write in clear, professional prose (not bullet points)
- Cite specific events or news
- Be concise (2-3 sentences max)
- Do NOT copy/paste news headlines

Answer:"""

    elif intent == 'trend_analysis':
        prompt = f"""You are a financial analyst. Based on the data below, analyze WHEN and HOW {ticker} has been trending.

{full_context}

User Question: {query}

Instructions:
- Answer the specific timing question asked
- Reference price movements and dates if available
- Be concise (2-3 sentences max)
- Write in clear prose, not headlines

Answer:"""

    elif intent == 'price_query':
        prompt = f"""Based on the stock data below, answer this price question about {ticker}.

{full_context}

User Question: {query}

Instructions:
- Give the specific price information requested
- Be direct and accurate
- Keep it brief (1-2 sentences)

Answer:"""

    elif intent == 'milestone_query':
        prompt = f"""Answer this question about {ticker} stock using the data below.

Current market cap: ${price_data.get('market_cap', 0):,.0f}

Question: {query}

News headlines:
{chr(10).join([f"- {a.get('title', '')}" for a in news_articles[:5]])}

Answer the question directly. If the milestone isn't mentioned in the news, say "No recent news about this milestone."

Answer:"""

    else:
        prompt = f"""Based on the stock data and news below, answer this question about {ticker}.

{full_context}

User Question: {query}

Instructions:
- Provide a helpful, accurate response
- Write in clear prose
- Be concise

Answer:"""

    # Get response from LLM
    response_text = smart_llm_invoke(prompt)

    # Add sources if news articles exist
    if news_articles and len(news_articles) > 0:
        response_text += "\n\n**Sources:**"
        for i, article in enumerate(news_articles[:5], 1):
            response_text += f"\n{i}. [{article.get('title', 'No title')}]({article.get('link', '#')}) - {article.get('source', 'Unknown')} ({article.get('date', 'No date')})"

    state['analysis'] = response_text

    return state

In [126]:
def build_workflow():
    workflow = StateGraph(AgentState)


    workflow.add_node("intent_classifier", intent_classifier)
    workflow.add_node("ticker_extractor", ticker_extractor)
    workflow.add_node("price_fetcher", price_fetcher)
    workflow.add_node("financial_fetcher", financial_fetcher)
    workflow.add_node("news_fetcher", news_fetcher)
    workflow.add_node("news_analyzer", news_analyzer)
    workflow.add_node("response_generator", response_generator)

    workflow.add_edge(START, "intent_classifier")
    workflow.add_edge("intent_classifier", "ticker_extractor")
    workflow.add_edge("ticker_extractor", "price_fetcher")
    workflow.add_edge("price_fetcher", "financial_fetcher")
    workflow.add_edge("financial_fetcher", "news_fetcher")
    workflow.add_edge("news_fetcher", "news_analyzer")
    workflow.add_edge("news_analyzer", "response_generator")
    workflow.add_edge("response_generator", END)

    return workflow.compile()
agent = build_workflow()

In [127]:
def fetch_currency(ticker: str):
    """
    Returns (currency_symbol, currency_code)
    """
    try:
        info = yf.Ticker(ticker).info
        currency_code = info.get("currency", "USD")
    except Exception:
        currency_code = "USD"

    SYMBOL_MAP = {
        "USD": "$", "INR": "₹", "EUR": "€", "GBP": "£",
        "JPY": "¥", "CNY": "¥", "HKD": "HK$",
        "AUD": "A$", "CAD": "C$", "CHF": "CHF",
        "KRW": "₩", "BRL": "R$", "ZAR": "R"
    }

    symbol = SYMBOL_MAP.get(currency_code, currency_code + " ")
    return symbol, currency_code


In [128]:
def query_stocks(user_query: str):
    initial_state = {
        'query': user_query,
        'ticker': '',
        'intent': '',
        'price_data': {},
        'financial_data': {},
        'news_articles': [],  # Start empty
        'news_context': [],
        'sentiment_score': 0.0,
        'analysis': '',
        'recommendation': '',
        'messages': []
    }

    result = agent.invoke(initial_state)

    return {
        'answer': result['analysis'],
        'ticker': result['ticker'],
        'sentiment': result['sentiment_score'],
        'debug_messages': result['messages']
    }

In [130]:
result1 = query_stocks("Is the stock price of BMW rising?" )
print(f"Query:Is the stock price of BMW rising?")
print(f"Answer: {result1['answer']}\n")

[NEWS ANALYZER DEBUG] Received 34 articles
Query:Is the stock price of BMW rising?
Answer: Yes, the stock price of BMW.BO is rising. The current price is ₹37.76, up from the previous close of ₹37.73.

**Sources:**
1. [Recycled Carbon Fiber Industry Outlook Report 2026: A $263.87 Million Market by 2033 | Astute Analytica](https://www.globenewswire.com/news-release/2026/01/22/3224202/0/en/Recycled-Carbon-Fiber-Industry-Outlook-Report-2026-A-263-87-Million-Market-by-2033-Astute-Analytica.html) - GlobeNewswire (2026-01-22)
2. [Indonesia Used Car Market Report 2026-2031: Odometer Fraud Persists, Highlighting Need for AI Inspections in Indonesia's Used Car Sector](https://www.globenewswire.com/news-release/2026/01/22/3223661/28124/en/Indonesia-Used-Car-Market-Report-2026-2031-Odometer-Fraud-Persists-Highlighting-Need-for-AI-Inspections-in-Indonesia-s-Used-Car-Sector.html) - GlobeNewswire (2026-01-22)
3. [Bulletproof Armored Vehicle Research Report 2026: $21.83 Bn Market Opportunities, Trends